In [ ]:
import numpy as np
import gudhi
import matplotlib.pyplot as plt
import matplotlib as mpl

import scipy.spatial.distance as dist
import itertools

import iblofunmatch.inter as ibfm
import iblofunmatch.topological_data_quality_0 as tdq0
output_dir = "output" 
plots_dir = "plots/example_computation/"

import os 
os.makedirs(output_dir, exist_ok=True)
os.makedirs(plots_dir, exist_ok=True)

In [ ]:
RandGen = np.random.default_rng(2)
# # Generate Random Sample
X = ibfm.sampled_circle(0,2,6, RandGen)
S_indices = RandGen.choice(X.shape[0],3, replace=False)
# Sort and save 
S_compl = np.ones(X.shape[0], dtype="bool")
S_compl[S_indices] = False
X = np.vstack((X[S_indices], X[S_compl]))
X[3] = [-0.1,0]
X[4] = [0.4,0]
X[5] = [0.15,np.sqrt(0.5**2 - 0.25**2)]
S_indices = range(len(S_indices))
np.savetxt("Z_example_new.txt", X, fmt="%.4f")
np.savetxt("X_idx_example_new.txt", S_indices, fmt="%d")
# Load X and S_indices from files 
X = np.loadtxt("Z_example_new.txt")
S_indices = np.loadtxt("X_idx_example_new.txt", dtype="int")
S = X[S_indices]
# Plot point cloud
fig, ax = plt.subplots(ncols=1, figsize=(3,3))
ax.scatter(S[:,0], S[:,1], color=mpl.colormaps["RdBu"](0.3/1.3), s=60, marker="o", zorder=2)
ax.scatter(X[:,0], X[:,1], color=mpl.colormaps["RdBu"](1/1.3), s=40, marker="x", zorder=1)
ax.set_axis_off()
plt.savefig(plots_dir + "points_0.png")

In [ ]:
# Compute matching using minimum spanning trees
filtration_list_X, pairs_arr_X = tdq0.filtration_pairs(X)
filtration_list_S, pairs_arr_S = tdq0.filtration_pairs(S)
F = tdq0.get_inclusion_matrix(pairs_arr_S, pairs_arr_X, S_indices)
matching = tdq0.get_inclusion_matrix_pivots(F, X.shape[0])

In [ ]:
pairs, multiplicities = tdq0.compute_mathing_diagram(filtration_list_S, filtration_list_X, matching)

In [ ]:
pairs

In [ ]:
multiplicities

In [ ]:
fig, ax = plt.subplots(figsize=(5,5))
pairs, multiplicities = tdq0.compute_mathing_diagram(filtration_list_S, filtration_list_X, matching)
tdq0.plot_matching_diagram(pairs, ax)
# ax.set_xticks(np.arange(0,2.6,0.5))
plt.savefig("matching_diagram_example.png")

# Stability plots

In [ ]:
RandGen = np.random.default_rng(5)
epsilon = 0.27
noise = RandGen.uniform(0,epsilon,2 * X.shape[0]).reshape((X.shape[0],2))
Xm = X + noise
Xm = Xm[:-2]
Xm = np.vstack((Xm, [0.27,0.23]))
Xm = np.vstack((Xm, Xm[2]-[epsilon/np.sqrt(2),epsilon/np.sqrt(2)]))
# Add a point to Xm
Xm = np.vstack((Xm, X[0]+[0.5*epsilon,-0.5*epsilon]))
S_indices_m = list(S_indices) + [Xm.shape[0]-2]
np.savetxt("Zm_example.txt", Xm, fmt="%.4f")
Xm = np.loadtxt("Zm_example.txt", dtype="float")
Sm = Xm[S_indices_m]

In [ ]:
Xm

In [ ]:
Sm

In [ ]:
fig, ax = plt.subplots(ncols=1, figsize=(3,3))
#Scatter points and their modified versions
ax.scatter(S[:,0], S[:,1], color=mpl.colormaps["RdBu"](1/1.3), s=20, marker="o", zorder=2)
ax.scatter(X[:,0], X[:,1], color=mpl.colormaps["RdBu"](1/1.3), s=15, marker="x", zorder=2)
ax.scatter(Sm[:,0], Sm[:,1], color=mpl.colormaps["RdBu"](0.3/1.3), s=20, marker="o", zorder=2)
ax.scatter(Xm[:,0], Xm[:,1], color=mpl.colormaps["RdBu"](0.3/1.3), s=15, marker="x", zorder=2)
# Draw connecting lines between points 
# xcoord = list(zip(X[:,0], Xm[:,0]))
# ycoord = list(zip(X[:,1], Xm[:,1]))
# for xline, yline in zip(xcoord, ycoord):
#     ax.plot(xline, yline, c="black", zorder=1)

for pt in Xm:
    close_pt = X[np.argmin(np.sum((X-pt)**2, axis=1))]
    path = np.vstack((pt, close_pt))
    ax.plot(path[:,0], path[:,1], c="black", zorder=1)
for pt in X:
    close_pt = Xm[np.argmin(np.sum((Xm-pt)**2, axis=1))]
    path = np.vstack((pt, close_pt))
    print(np.sqrt(np.sum((path[0]-path[1])**2)))
    ax.plot(path[:,0], path[:,1], c="black", zorder=1)
ax.set_axis_off()
plt.savefig(plots_dir + "points_noise_example.png")

In [ ]:
filtration_list_Xm, pairs_arr_Xm = tdq0.filtration_pairs(Xm)
filtration_list_Sm, pairs_arr_Sm = tdq0.filtration_pairs(Sm)
F_m = tdq0.get_inclusion_matrix(pairs_arr_Sm, pairs_arr_Xm, S_indices_m)
matching_m = tdq0.get_inclusion_matrix_pivots(F_m, Xm.shape[0])
pairs_m, multiplicities_m = tdq0.compute_mathing_diagram(filtration_list_Sm, filtration_list_Xm, matching_m)

In [ ]:
matching_m

In [ ]:
pairs_m

In [ ]:
fig, ax = plt.subplots(figsize=(4,4))
pairs, multiplicities = tdq0.compute_mathing_diagram(filtration_list_S, filtration_list_X, matching)
max_x = np.max(pairs[pairs[:,0]<np.inf])
tdq0.plot_matching_diagram(pairs, ax, colorpt=mpl.colormaps["RdBu"](1/1.3), marker="o", size=20, hmax=max_x)
tdq0.plot_matching_diagram(pairs_m, ax, colorpt=mpl.colormaps["RdBu"](0.3/1.3), marker="x", size=25, hmax=max_x)
fin_pairs = pairs[pairs[:,0]<np.infty]
fin_pairs_m = pairs_m[pairs_m[:,0]<np.infty]
for pair, pair_m in zip(fin_pairs, fin_pairs_m[1:]):
    ax.plot([pair[0], pair_m[0]], [pair[1], pair_m[1]], c="black", zorder=1)

plt.savefig(plots_dir + "matching_diagram_stable.png")

In [ ]:
pairs

In [ ]:
pairs_m

In [ ]:
pairs_c = np.vstack((pairs,pairs[-1]))

In [ ]:
np.abs(pairs_c-pairs_m[1:])